<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/ACL_U_Project_Part1_Combine_Data_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this Jupyter notebook is to combine and split the data from the various source files we were provided. The current format is extremely redundant, will cause memory issues if handled directly in its entirety. By combining and splitting the data we will have a much easier time working with the dataset.

This notebook splits the Combined input into the following files:


*   dfMerged.csv - Includes all voting information (need to rename)
*   dfVoters.csv - Includes information with regard to the voters
*   dfOutreach.csv - Includes information based on which outreach effort(s) affected a particular voter.



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import requests

if 'google.colab' in str(get_ipython()):
  #We keep changing the different paths, so I'm going to check if various configurations exist.
  if (os.path.isdir("/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/ACLU/")):
    GoogleDriveBase = "/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/" #Anupriya
    WorkingDirectory = GoogleDriveBase + 'ACLU/' #Anupriya
  elif (os.path.isdir("/content/drive/MyDrive/Projects/ACLU")):
    GoogleDriveBase = "/content/drive/MyDrive/" #Mackenzie
    WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Mackenzie
  else:  
    if (os.path.isdir("/content/drive/My Drive/Projects/ACLU/")):
      GoogleDriveBase = "/content/drive/My Drive/" #Kyle
      WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Kyle
else: # We're not running in Google Colab, which means we're probably running locally. 
  #Put code here for local copies of the files
  GoogleDriveBase = "" 
  WorkingDirectory = GoogleDriveBase + "" 


WorkingFiles = WorkingDirectory + 'WorkingFiles/'
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"


#Make the necessary folders for the script to run.
ListOfAllRequiredDirectories = [WorkingDirectory + 'Pickle', 
                          WorkingDirectory + 'WorkingFiles',
                          WorkingDirectory + 'AdditionalData',
                          WorkingDirectory + 'ACLUData']

for folder in ListOfAllRequiredDirectories:
  RunningPath = GoogleDriveBase + folder + "/"
  Path(RunningPath).mkdir(parents=True, exist_ok=True)

In [3]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import csv
import string
import pandas as pd
import gc
import re
import string

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

tar: spark-2.4.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [5]:
#https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
if os.path.exists("spark-3.1.2-bin-hadoop3.2.tgz") == False:
  !wget -q https://apache.claz.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz #This has failed a few times, you may need to use a mirror from another site: https://www.apache.org/dyn/closer.lua/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar -xvzf spark-3.1.2-bin-hadoop3.2.tgz



spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

In [6]:
import findspark

!ls
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.memory", "12g") \
        .config("spark.driver.maxResultSize", "5g")\
        .getOrCreate()

spark

drive  sample_data  spark-3.1.2-bin-hadoop3.2  spark-3.1.2-bin-hadoop3.2.tgz


In [7]:
#If you want to see the spark UI
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')
#!curl -s http://localhost:4040/api/tunnels

In [8]:
AllFilesInTheSubfolder = []
for subfolder in os.walk(WorkingDirectory + "ACLUData"):
  #Skip the pickle folder
  if "Pickle" in subfolder[0] or "WorkingFiles" in subfolder[0]:
      continue
  for file in subfolder[2]:
    FileWithPath = subfolder[0] +"/"+ file
    match = re.search("(.*)(text|mail|phone|call|postcards)", file.lower())
    if match == None:
      raise ValueError("Could not parse: " + file + "\n Please take a look at the type of communication and ensure that it's on the list. \nThis should only happen when we add new files whose name is inconsistant with the older files.")
    CommunicationType = match.group(2).replace("_", " ").strip()
    #We have some inconsistant communication type names. So, I'm doing a little cleanup here.
    if CommunicationType == "call":
      CommunicationType = "phone"
    if CommunicationType == "postcards":
      CommunicationType = "mail"
    FileWithDetails = (FileWithPath, match.group(1).replace("_", " ").strip(), match.group(2).replace("_", " ").strip(), file)
    AllFilesInTheSubfolder.append(FileWithDetails)

    #print(FileWithDetails)

In [9]:
len(AllFilesInTheSubfolder)

25

In [10]:
from pyspark.sql import functions as F
columns = []

firstDataFrame = None

for file in AllFilesInTheSubfolder:
  df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load(file[0])
  df = df.withColumn("ElectionType", F.lit(file[1]))
  df = df.withColumn("CommunicationType", F.lit(file[2]))
  df = df.withColumn("File", F.lit(file[3]))
  columns.extend(df.columns)
  if firstDataFrame is None:
    firstDataFrame = df
  else:
    FirstNotSelected = set(firstDataFrame.columns) - set(df.columns)
    print("FirstNotSelected")
    for value in FirstNotSelected:
      print(value)
      df = df.withColumn(value, F.lit(""))


    print("SecondNotSelected")
    SecondNotSelected = set(df.columns) - set(firstDataFrame.columns)
    for value in SecondNotSelected:
      print(value)
      firstDataFrame = firstDataFrame.withColumn(value, F.lit(""))
    firstDataFrame = firstDataFrame.union(df)
  #Count takes a bit
  #print(firstDataFrame.count())
columns = set(columns)

FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
SecondNotSelected
FirstNotSelected
Zip544
State43
Zip445
City42
SecondNotSelected
Zip446
City43
VotedStatus
Zip545
State44
FirstNotSelected
Zip446
City43
VotedStatus
Zip545
State44
SecondNotSelected
FirstNotSelected
Zip544
State43
Zip445
City42
SecondNotSelected
FirstNotSelected
Zip544
State43
Zip445
City42
SecondNotSelected
FirstNotSelected
Zip544
State43
Zip445
City42
SecondNotSelected
FirstNotSelected
Zip544
State43
Zip445
City42
SecondNotSelected
FirstNotSelected
Zip446
City43
VotedStatus
Zip545
State44
SecondNotSelected
FirstNotSelected
Zip446
City43
VotedStatus
Zip545
State44
SecondNot

In [11]:
print(firstDataFrame.count())
len(AllFilesInTheSubfolder)


3196831


25

In [12]:
#firstDataFrame.show()

In the following section we will be pulling out all unique voter information.

Please Note: As this data includes real world PII we will be making an effort to never show the data from this dataframe.

In [13]:
#It looks like "Voter File VANID" is a unique key for each user, so I'm going to pull out any other person information and stick that into another dataframe. This will reduce memory footprint, which will be necessary for the pivot
#that's coming up.
import pyspark.sql.functions as func
from pyspark.sql.functions import row_number
from pyspark.sql import DataFrameWriter
from pyspark.sql.window import Window

dataframesbytypejoinedwithoutidvalues = {}

id_columns = ["Voter File VANID", "FirstName", "MiddleName", "LastName", "Address", "DWID",  'Age', 'Primary19', 'MaritalStatus', 'mCity', 'Mass_Incarceration', 'StreetType', 'StreetSuffix',
  'mAddress', 'mState', 'mZip5', 'mZip4', 'Sex', 'Suffix', 'CD', 'SD', 'HD', 'PreferredEmail', 'Preferred Phone', 'Cell Phone', 'CountyName', 'DOB', 'DateReg', 'Home Phone', 
  'EthnicCatalistName', 'Party', 'PersonalEmail', 'RaceName', 'StreetNo', 'StreetNoHalf', 'StreetPrefix', 'StreetName', 'AptType', 'AptNo', 
  '2020_Biden_Support', 'Voting_Aug_Prim', 'PoliceAccountability', 'VBM_Application', 'MarijuanaConviction',  'Absentee_Voting'] #, 'VotedStatus', 'Voting_Nov_Gen','Zip5', 'City', 'State', 'Zip4', 'City.1', 'State.1', 'Zip5.1', 'Zip4.1', 
primary_key = ["Voter File VANID"]
dfVoters = None

dfVoters = firstDataFrame.select(id_columns)
dfVoters = dfVoters.na.fill(0)

WindowSpec  = Window.partitionBy("Voter File VANID").orderBy("Age")
dfVoters = dfVoters.withColumn("row_number",row_number().over(WindowSpec))
dfVoters = dfVoters.where(dfVoters['row_number'] == 1)

print(dfVoters.count())
#print(type(a))

#dfVoters.show()




1633349


In [14]:
#dfVoters.toPandas().to_csv(WorkingFiles + "dfVoters.csv", encoding='utf-8')
#dfVoters.write.csv(WorkingFiles + "dfVoters.csv", mode="overwrite")
import glob
import os
import shutil

def SaveSingleFile(df, fileName):
  tempDir = WorkingFiles + "temp"
  if not os.path.exists(tempDir):
    os.makedirs(tempDir)
  else:
    shutil.rmtree(tempDir)
    os.makedirs(tempDir)
  print("Writing File")
  df.coalesce(1).write.csv(WorkingFiles + "temp", mode="overwrite", header=True)
  #df.write.csv(WorkingFiles + "temp", mode="overwrite", header=True)
  os.chdir(WorkingFiles + "temp")
  print("Moving file")
  for file in glob.glob('part-00000-*'):
      shutil.move(file, fileName)



In [15]:

SaveSingleFile(dfVoters, WorkingFiles + "dfVotersSingle.csv")


Writing File
Moving file


Finally, we will save the data to disk for use by the next Jupyter notebook.

[Merge Data](https://colab.research.google.com/drive/1epgGr9YFq1l3BMvDLApkWXnZ9TjCcJCo?usp=sharing)

In [16]:
ElectionColumns = ["Voter File VANID", "ElectionType", "CommunicationType", "File"]

RemainingColumns = list(set(firstDataFrame.columns) - (set(id_columns + ElectionColumns)))
dfMerged = firstDataFrame.select(RemainingColumns)
#dfMerged.coalesce(1).write.csv(WorkingFiles + "dfMerged.csv", mode="overwrite")

print(RemainingColumns)
print(dfMerged.show())

['Voting_Nov_Gen ', 'State43', 'Primary04', 'Special98', 'Primary12', 'Primary02', 'Primary20Party', 'General11', 'Special02', 'Primary20', 'Primary12Party', 'Primary09Party', 'Special03', 'Municipal13', 'PresPrimary12', 'General19', 'Primary16', 'Special16', 'General09', 'General98', 'Special99', 'Primary14', 'Special08', 'General04', 'Primary10', 'Special11', 'VotedStatus', 'City9', 'Zip544', 'Special15', 'PresPrimary08Party', 'Special00', 'General18', 'Primary11Party', 'Special07', 'Primary98', 'PresPrimary16Party', 'General20', 'StateFileID', 'Special19', 'Primary17', 'Special17', 'Primary08Party', 'PresPrimary08', 'Primary17Party', 'General15', 'General10', 'PresPrimary20', 'PresPrimary16', 'State10', 'Primary06', 'Primary00', 'State44', 'Primary15', 'Special01', 'Zip446', 'General02', 'PresPrimary20Party', 'Primary16Party', 'Special13', 'Primary19Party', 'Zip545', 'Primary18Party', 'Primary10Party', 'General00', 'Primary14Party', 'Special05', 'Special12', 'Zip511', 'General06', '

In [17]:
SaveSingleFile(dfMerged, WorkingFiles + "dfMerged.csv")


Writing File
Moving file


In [ ]:
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
from pyspark.sql.functions import udf

Bucketized = spark.read.option("header",True).csv(WorkingFiles + "dfMergedSingle.csv")

dfVoters_sub = df[['Voter File VANID', 'Age', 
       'MaritalStatus', 'Sex', 'CountyName',
       'Party', 'RaceName', 'Primary19', '2020_Biden_Support', 
       'PoliceAccountability','VBM_Application', 'MarijuanaConviction','Absentee_Voting', 'Mass_Incarceration']]

bins= [18.0,29.0,44.0,64.0, 200.0]
labels = ['18-29','30-44','45-64','65+']

firstDataFrame = firstDataFrame.withColumn("Age", firstDataFrame["Age"].cast(IntegerType()))
Bucketized = Bucketizer().setInputCol("Age").setOutputCol( "age_bucket").setSplits( bins ).transform(firstDataFrame)#.collect()


#dfVoters_sub['split'] = pd.cut(dfVoters_sub['Age'], bins=bins, labels=labels, right=True)

t = {'0.0':'18-29', '1.0': '30-44', '2.0':'45-64', '3.0': '65+'}
#udf_foo = udf(lambda x: t[x], StringType())
#Bucketized = Bucketized.withColumn("AgeGroup", udf_foo("age_bucket"))

#Bucketized.withColumn("DATE", F.when(F.col("DATE").isNull(), '').otherwise(F.col("DATE"))).show()

#Bucketized = Bucketized.drop("age_bucket")
#Bucketized = Bucketized.drop("AgeGroup")

Bucketized = Bucketized.withColumn("AgeGroup", Bucketized["age_bucket"].cast("string"))
Bucketized = Bucketized.replace(t, "AgeGroup")

print(Bucketized.show())
SaveSingleFile(dfVoters_sub, WorkingFiles + "dfVoters_sub.csv")

In [ ]:
Bucketized = spark.read.option("header",True).csv(WorkingFiles + "Bucketized.csv")

In [ ]:
VoterColumns = ["Voter File VANID", "FirstName", "MiddleName", "LastName", "Address", "DWID",  'Age', 'Primary19', 'MaritalStatus', 'mCity', 'Mass_Incarceration', 'StreetType', 'StreetSuffix',
  'mAddress', 'mState', 'mZip5', 'mZip4', 'Sex', 'Suffix', 'CD', 'SD', 'HD', 'PreferredEmail', 'Preferred Phone', 'Cell Phone', 'CountyName', 'DOB', 'DateReg', 'Home Phone', 
  'EthnicCatalistName', 'Party', 'PersonalEmail', 'RaceName', 'StreetNo', 'StreetNoHalf', 'StreetPrefix', 'StreetName', 'AptType', 'AptNo', 
  '2020_Biden_Support', 'Voting_Aug_Prim', 'PoliceAccountability', 'VBM_Application', 'MarijuanaConviction',  'Absentee_Voting', 'age_bucket', 'AgeGroup',
  'Zip545', 'State43', 'Zip412', 'StateFileID', 'Zip511', 'City42', 'Ballot_Application', 'Zip446', 'Zip445', 'Zip544'] 

OutreachColumns = ["Voter File VANID", "ElectionType", "CommunicationType", "File"]

OtherColumns = ['Voting_Nov_Gen ',  '2020_Biden_Support']

VoterColumnsToRemove = list(set(VoterColumns) - set(primary_key))
OutreachColumnsToRemove = list(set(OutreachColumns) - set(primary_key))
OtherColumnsToRemove = list(set(OtherColumns) - set(primary_key))


AllColumns = set(Bucketized.columns)
RemainingColumns = list(((set(Bucketized.columns) - set(VoterColumnsToRemove)) - set(OtherColumnsToRemove)) - set(ElectionColumns))


VotingColumns = list(set(primary_key + RemainingColumns))


In [ ]:
dfVoters = Bucketized.select(VoterColumns)
#print(dfVoters.show())
#dfVoters.write.csv(WorkingFiles + "dfVoters.csv", mode="overwrite")
SaveSingleFile(dfVoters, WorkingFiles + "dfVoters.csv")

In [ ]:
#Pull out the file information, this indicates which outreach efforts they were included in.
dfOutreach = Bucketized.select(ElectionColumns)
#dfOutreach.write.csv(WorkingFiles + "dfOutreach.csv", mode="overwrite")

#SaveSingleFile(dfMerged, WorkingFiles + "dfOutreachSingle.csv")
dfOutreach.show()

In [ ]:
dfRemaining = Bucketized.select(VotingColumns)
#dfRemaining.show()

In [ ]:
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable 

primary_key = ["Voter File VANID"]
columns_processed = []

def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))
    #print(_vars_and_vals)
    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

def renameColumns(df, ColumnList1, ColumnList2):
  for i in range(len(ColumnList1)):
    df = df.withColumnRenamed(ColumnList1[i], ColumnList2[i])
  return df

def ReshapeDataframeYears(df):
  ColumnsToTransform = ["Special##", "General##", "Primary##", "PresPrimary##", "PresPrimary##Party", "Primary##Party", "Municipal##"]
  #ColumnsToTransform = ["Special##"]
  melted_dataframes = []
  runningDataFrame = None

  for name in ColumnsToTransform:
    ColumnsForName = []
    years = []
    number_index = name.index("##")
    for column in df.columns:
      if len(column) < len(name):
        continue
      first = column[0:number_index]
      name_first = name[0:number_index]
      number = column[number_index:number_index+2]
      if len(column) - number_index+2 == 0:
        second = ""
        name_second = ""
      else:
        second = column[number_index+2:len(column)]
        name_second = name[number_index+2:len(name)]

      if first == name_first and second == name_second:
        ColumnsForName.append(column)
        columns_processed.append(column)
        year = column[len(first):len(first)+2]

        if int(year) < 50:
          year = "20" + year
        else:
          year = "19" + year
        years.append(year)
        #print(int(year))
    df_reduced = df.select(primary_key + ColumnsForName)
    #print("DFReduced " + str(type(df_reduced)))
    #df_reduced.withColumnRenamed(existing = list((primary_key + ColumnsForName)), new = list((primary_key + years)))
    #df_reduced.columns = primary_key + years
    #df_melted = melt(df_reduced, id_vars=primary_key, value_vars=years)
    
    df_reduced = renameColumns(df_reduced, (primary_key + ColumnsForName), (primary_key + years))
    df_melted = melt(df_reduced, id_vars=primary_key, value_vars=(years))
    
    #Restructure the columns
    final_columns = primary_key.copy()
    final_columns.append("year")
    final_columns.append("participation")
    #print(df_melted.show())
    df_melted = renameColumns(df_melted, df_melted.columns, final_columns)
    df_melted = df_melted.withColumn("ElectionType", F.lit(name))
    #Drop records where we don't have participation data. 
    df_melted = df_melted.na.drop()
    #print(df_melted.head())
    melted_dataframes.append(df_melted)
    if runningDataFrame is None:
      runningDataFrame = df_melted
    else:
      runningDataFrame = runningDataFrame.union(df_melted)
  return runningDataFrame

def CombineMeltedDataframes(melted_dataframes):
  #Now join them all back together
  #Rather silly that this is my method, but I tried a few different versions to find the most memory efficient way to do this
  return pd.concat(melted_dataframes)

melteddfs = ReshapeDataframeYears(dfRemaining)
#ColumnsThatWereNotPartOfTheRestructuredColumns = set(Bucketized.columns) - set(columns_processed)
#melteddfs.show()

In [ ]:
#melteddfs.write.csv(WorkingFiles + "dfreshapedUnique.csv", mode="overwrite")
SaveSingleFile(melteddfs, WorkingFiles + "dfreshapedUnique.csv")

In [ ]:
#Put Togther Voter Information DataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import upper, col

AllRegisteredVoters = spark.read.option("header",True).csv(WorkingDirectory + 'AdditionalData/All Registered Voters Michigan.csv')
AllRegisteredVoters = AllRegisteredVoters.withColumnRenamed('County', 'CountyName')
ACLUVotersGrouped = dfVoters.groupby('CountyName').count()
ACLUVotersJoineWithAllVotersInfo = AllRegisteredVoters.join(ACLUVotersGrouped, upper(dfVoters['CountyName']) == AllRegisteredVoters['CountyName'], 'inner')
ACLUVotersJoineWithAllVotersInfo = ACLUVotersJoineWithAllVotersInfo.withColumn("fraction", (F.col("count") / F.col("All Registered Voters")))

#total_voter_pop.to_csv("total_voter_pop.csv")
ACLUVotersJoineWithAllVotersInfo.show()

SaveSingleFile(ACLUVotersJoineWithAllVotersInfo ,WorkingFiles+ "total_voter_pop.csv")

In [ ]:


def SummarizeForValue(df, ColumnsWeWantValuesFor, FileName):
  SortingColumns = ['RaceName', 'Sex', 'AgeGroup', 'CountyName']
  selected = df.select(SortingColumns + primary_key + ColumnsWeWantValuesFor)
  selected = selected.groupBy(SortingColumns + ColumnsWeWantValuesFor).count()
  print(selected.show())
  SaveSingleFile(selected, WorkingFiles + FileName)

print(Bucketized.columns)
SummarizeForValue(Bucketized, ['CommunicationType', 'ElectionType'], "AggregateByCommunication.csv");
SummarizeForValue(Bucketized, ['Mass_Incarceration'], "mass_inc.csv");
SummarizeForValue(Bucketized, ['MarijuanaConviction'], "weed.csv");
#SummarizeForValue(['election_votes'], "weed.csv");


#df_aggregate.to_csv("aggregate.csv")

#biden_support_df.to_csv("biden_support_df.csv")
#mass_inc.to_csv("mass_inc.csv")
#policing.to_csv("policing.csv")
#weed.to_csv("weed.csv")
#election_votes.to_csv("election_votes.csv")
#plotly_fips.to_csv("plotly_fips.csv")


In [ ]:
dfVoters_With_Election = dfVoters.join(melteddfs, dfVoters['Voter File VANID'] == melteddfs['Voter File VANID'], 'inner').drop(dfVoters['Voter File VANID'])
dfVoters_With_Election = dfVoters_With_Election.withColumnRenamed("ElectionType", "ElectionCategory")
dfVoters_With_Election_and_outreach = dfVoters_With_Election.join(dfOutreach, dfVoters_With_Election['Voter File VANID'] == dfOutreach['Voter File VANID'], 'inner')





In [ ]:
dfVoters_With_Election_and_outreach.show()

In [ ]:
dfVoters_count = dfVoters_With_Election_and_outreach.groupBy( "AgeGroup", 'CountyName', 'RaceName', 'Sex', '2020_Biden_Support', 'MarijuanaConviction', 'PoliceAccountability', 'year', 'participation', 'ElectionCategory', 'ElectionType', 'CommunicationType').count()
dfVoters_count.show()

In [ ]:
SaveSingleFile(dfVoters_With_Election, WorkingFiles + "dfVoters_With_Election.csv")